In [1]:
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
from PyQt5 import *
#from deepface import DeepFace 
import sys
import os
import cv2

In [2]:
class QComboBox(QtWidgets.QComboBox):
    def __init__(self, parent=None):
        super(QIComboBox, self).__init__(parent)
        
class Wizard(QtWidgets.QWizard):
    def __init__(self, parent=None):
        super(Wizard, self).__init__(parent)

        #add pages - these are where conditions will go
        page1 = Page1()
        self.addPage(page1)
        
        page2 = Page2()
        self.addPage(page2)
        
        page3single = self.addPage(Page3Single())
        page3batch = self.addPage(Page3Batch())
        page2.nextId = lambda: page3batch if page2.batch_cb.isChecked() else page3single
        # this isn't reliable and causes the unneccessary creation of page3single even if batch is selected.
        # also won't work two times in a row - how do you get the ID of EITHER page3single OR page3batch??
        
        page4 = Page4Single()
        self.addPage(page4)
        
        self.setWindowTitle("DCiFR")
        self.setGeometry(0, 0, 800, 600)
        
class Page1(QtWidgets.QWizardPage):
    def __init__(self, parent=None):
        super(Page1, self).__init__(parent)
        
        self.title_label = QLabel('Welcome to DCiFR!', self)
        self.title_label.move(50, 30)
        self.title_label.setFont(QFont('Arial', 20))
        self.title_label.adjustSize()

        self.title_label = QLabel('Attributes', self)
        self.title_label.move(100, 75)
        self.title_label.setFont(QFont('Arial', 15))
        self.title_label.adjustSize()

        #hover info 
        info = QLabel('Check the boxes that apply. Hover for more info!', self)
        info.move(50, 125)
        info.setFont(QFont('Arial', 10))
        info.adjustSize()
        myFont=QtGui.QFont()
        myFont.setItalic(True)
        info.setFont(myFont)
        info.adjustSize()
        
        #Check boxes
        age_cb = QCheckBox('Age', self)
        age_cb.move(50, 150)
        race_cb = QCheckBox('Race', self)
        race_cb.move(50, 200)
        age_cb.adjustSize()
        race_cb.adjustSize()
        gender_cb = QCheckBox('Gender', self)
        gender_cb.move(50, 250)
        emotion_cb = QCheckBox('Emotion', self)
        emotion_cb.move(50, 300)
        gender_cb.adjustSize()
        emotion_cb.adjustSize()
        
        #Adding slider for race attribute
        self.sld = QSlider(Qt.Horizontal, self)
        self.sld.setRange(0, 100)
        self.sld.move(500, 200)
        label = QLabel('Please select your threshold value \n(only applicable if you select the race attribute)', self)
        label.move(300, 150)
        self.sld.setToolTip('This is a slider for the threshold of the race attribute.')

        #Hovers
        age_cb.setToolTip('Check this box if you would like to analyze the age of the subject in your image(s)')
        race_cb.setToolTip('Check this box if you would like to analyze the race of the subject in your image(s)')
        gender_cb.setToolTip('Check this box if you would like to analyze the gender of the subject in your image(s)')
        emotion_cb.setToolTip('Check this box if you would like to analyze the emotion of the subject in your image(s)')
        
class Page2(QtWidgets.QWizardPage):
    def __init__(self, parent=None):
        super(Page2, self).__init__(parent)
        self.title_label = QLabel('DCiFR', self)
        self.title_label.move(50, 30)
        self.title_label.setFont(QFont('Arial', 20))
        self.title_label.adjustSize()
        
        self.title_label = QLabel('Single or Batch Mode', self)
        self.title_label.move(100, 75)
        self.title_label.setFont(QFont('Arial', 15))
        self.title_label.adjustSize()
        #hover info 
        info = QLabel('Check the box that applies. Hover for more info! Please select ONLY ONE option.', self)
        info.move(50, 125)
        info.setFont(QFont('Arial', 10))
        info.adjustSize()
        myFont=QtGui.QFont()
        myFont.setItalic(True)
        info.setFont(myFont)
        info.adjustSize()
        
        #Check boxes - hbox allows these to be exclusive
        hbox = QHBoxLayout()
    
        self.single_cb = QCheckBox('Single Image', self)
        self.single_cb.move(50, 150)
        self.batch_cb = QCheckBox('Batch Mode', self)
        self.batch_cb.move(50, 200)
        self.single_cb.adjustSize()
        self.batch_cb.adjustSize()
        
        group = QButtonGroup(self)
        group.addButton(self.single_cb)
        group.addButton(self.batch_cb)
        
        hbox.addWidget(self.single_cb)
        hbox.addWidget(self.batch_cb)
        
        #Hovers
        self.single_cb.setToolTip('Check this box if you would like to analyze demograhpics for a single image')
        self.batch_cb.setToolTip('Check this box if you would like to analyze demograhpics for more than one image')

class Page3Single(QtWidgets.QWizardPage):
    #upload and analyze a single image
    
    def detect_face_show(fpath):
        img = cv2.imread(fpath)
        if img is None:
            return(0)
        else:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            faces = face_cascade.detectMultiScale(gray, 1.1, 4)
            face_num = len(faces)
            if (face_num == 1):
                return DeepFace.analyze(faces, ['age', 'gender', 'race'], enforce_detection=False)
            else:
                return(0)
    
    def get_image_file(self):
            dialog = QFileDialog()
            file_name, _ = dialog.getOpenFileName(self, 'Open file', 'c:\\', "Image files (*.jpg)")
            #detect_face_show(file_name)
        
    def __init__(self, parent=None):
        super(Page3Single, self).__init__(parent)
        self.title_label = QLabel('DCiFR', self)
        self.title_label.move(50, 30)
        self.title_label.setFont(QFont('Arial', 20))
        self.title_label.adjustSize()
        self.title_label = QLabel('Upload Your Image Below', self)
        self.title_label.move(100, 75)
        self.title_label.setFont(QFont('Arial', 10))
        self.title_label.adjustSize()
        
        self.button1 = QPushButton("Select An Image to Upload Here", self)  
        self.button1.clicked.connect(self.get_image_file) 
        self.button1.move(50, 150)
        
class Page3Batch(QtWidgets.QWizardPage):
    #upload and analyze multiple images
    
    def get_image_files(self):
        
        #why does this not allow for several images to be selected at once?
        dialog = QFileDialog()
        dialog.setOption(dialog.DontUseNativeDialog, True)
        dialog.setFileMode(QFileDialog.ExistingFiles)
        file_name, _ = dialog.getOpenFileName(self, 'Open file', 'c:\\', "Image files (*.jpg)")
        print(file_name)
        #detect_face_show(file_name)
        
    def __init__(self, parent=None):
        super(Page3Batch, self).__init__(parent)
        self.title_label = QLabel('DCiFR', self)
        self.title_label.move(50, 30)
        self.title_label.setFont(QFont('Arial', 20))
        self.title_label.adjustSize()
        self.title_label = QLabel('Upload Your Images Below', self)
        self.title_label.move(100, 75)
        self.title_label.setFont(QFont('Arial', 10))
        self.title_label.adjustSize()
        
        self.button1 = QPushButton("Select Your Images to Upload Here", self)   
        self.button1.clicked.connect(self.get_image_files)
        self.button1.move(50, 150)
        
class Page4Single(QtWidgets.QWizardPage):
    def __init__(self, parent=None):
        super(Page4Single, self).__init__(parent)
        
        self.title_label = QLabel('DCiFR', self)
        self.title_label.move(50, 30)
        self.title_label.setFont(QFont('Arial', 20))
        self.title_label.adjustSize()

        self.title_label = QLabel('Results', self)
        self.title_label.move(100, 75)
        self.title_label.setFont(QFont('Arial', 15))
        self.title_label.adjustSize()
        
class Page4Batch(QtWidgets.QWizardPage):
    def __init__(self, parent=None):
        super(Page4Batch, self).__init__(parent)
        
        self.title_label = QLabel('DCiFR', self)
        self.title_label.move(50, 30)
        self.title_label.setFont(QFont('Arial', 20))
        self.title_label.adjustSize()

        self.title_label = QLabel('Results', self)
        self.title_label.move(100, 75)
        self.title_label.setFont(QFont('Arial', 15))
        self.title_label.adjustSize()
        
if __name__ == '__main__':
    app = QtWidgets.QApplication(sys.argv)
    wizard = Wizard()
    wizard.show()
    app.exec_()    
    
#TO-DO:

# work on connecting to deepface
# page 4? -> change page formatting for checkbox states
# multiple image upload is wonky